In [3]:
# Python natives
import os

os.chdir("/home/tim/Development/OCPPM/")
import lightgbm as lgb
import pprint
import numpy as np
import pandas as pd
import re
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from utilities import evaluation_utils

In [15]:
prediction_task = "regression"
if prediction_task == "regression":
    regression = True
elif prediction_task == "classification":
    regression = False
target_name = {
    "regression": "@@object_lifecycle_duration",
    "classification": "event_ea4",
}
eft_in_file = "data/CS/feature_encodings/baselines/EFT/eft.csv"

In [13]:
events_df = pd.read_csv(eft_in_file, sep=";")  # , dtype=eft_dtypes)
events_df.head()

,event_preceding_activitiesFinishTask,event_preceding_activitiesUpdateObjectState,event_preceding_activitiesReallocateTask,event_preceding_activitiesCreateObject,event_preceding_activitiesPerformAction,event_preceding_activitiesAttachDocument,event_preceding_activitiesLinkObjects,event_preceding_activitiesSaveObject,event_preceding_activitiesAttachNote,event_elapsed_time,...,event_resourceMulti,event_ea1,event_ea2,event_ea3,event_ea4,event_ea6,event_ea8,event_ea10,event_ea12,event_ea14
0,-0.689369,-0.457546,-0.427891,-0.137938,-0.226369,-0.221569,-0.188439,5.364903,-0.109962,0.342960,...,-0.472678,-0.069109,-0.126739,-0.438403,-1.012772,0.510668,0.068685,0.309297,-0.580952,-0.455247
1,-0.689369,-0.457546,-0.427891,-0.137938,-0.226369,-0.221569,-0.188439,5.364903,-0.109962,1.105819,...,-0.472678,-0.069109,-0.126739,-0.438403,-1.012772,0.510668,0.068685,0.309297,-0.580952,-0.455247
2,-0.689369,-0.457546,-0.427891,-0.137938,-0.226369,-0.221569,-0.188439,5.364903,-0.109962,0.472925,...,-0.472678,-0.069109,-0.126739,-0.438403,-1.012772,0.510668,0.068685,0.309297,-0.580952,-0.455247
3,-0.689369,-0.457546,-0.427891,-0.137938,-0.226369,-0.221569,-0.188439,5.364903,-0.109962,1.328340,...,-0.472678,-1.134898,-0.126739,-0.438403,-1.012772,-0.897602,0.068685,0.309297,-0.580952,-0.455247
4,-0.689369,-0.457546,-0.427891,-0.137938,-0.226369,-0.221569,-0.188439,5.364903,-0.109962,2.556901,...,-0.472678,-0.069109,-0.126739,-0.438403,-0.226463,0.510668,0.068685,0.309297,-0.007038,-0.455247


In [16]:
# make train test split
X, y = (
    events_df.drop(columns=target_name.values()),
    events_df.loc[:, target_name[prediction_task]],
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=42
)

train_data = lgb.Dataset(
    X_train,
    label=y_train,
)
valid_data = lgb.Dataset(
    X_valid,
    label=y_valid,
)

In [17]:
if not regression:
    events_df[target_name[prediction_task]].value_counts(normalize=True).sort_index()

In [18]:
params = {
    "metric": ["multi_logloss"],
    "num_boost_round": 100,
    "stopping_rounds": 100,
    "num_threads": 4,
}
if regression:
    params |= {"objective": "regression", "metric": ["mse", "mae", "mape", "rmse"]}
else:
    params |= {"objective": "multiclass", "num_class": 7}


bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(params["stopping_rounds"])],
)

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.398713 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 444
[LightGBM] [Info] Number of data points in the train set: 1227098, number of used features: 24
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Info] Start training from score -0.001301
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.784935	valid_0's l1: 0.566938	valid_0's mape: 0.434571	valid_0's rmse: 0.885966


In [19]:
if regression:
    y_train_preds = bst.predict(X_train)
    y_valid_preds = bst.predict(X_valid)
else:
    y_train_preds = np.apply_along_axis(
        evaluation_utils.get_preds_from_probs, axis=1, arr=bst.predict(X_train)
    )
    y_valid_preds = np.apply_along_axis(
        evaluation_utils.get_preds_from_probs, axis=1, arr=bst.predict(X_valid)
    )

In [21]:
# Run model evaluation, TODO: store this somewhere (in JSON)
eval_train = evaluation_utils.get_evaluation(
    y_train, y_train_preds, regression=regression
)
eval_valid = evaluation_utils.get_evaluation(
    y_valid, y_valid_preds, regression=regression
)
experiment_settings = {"experiment_settings": params}
evaluation_report = {"train": eval_train, "validation": eval_valid}
pprint.pprint(evaluation_report)
pprint.pprint(experiment_settings)

{'train': {'report': {'MAE': 0.5656807604209542,
                      'MAPE': 1.4912092269186632,
                      'MSE': 0.7793908056898542,
                      'R^2': 0.21273217624286378}},
 'validation': {'report': {'MAE': 0.5669379751553116,
                           'MAPE': 1.4747371363691022,
                           'MSE': 0.7849352810605719,
                           'R^2': 0.20831684778502646}}}
{'experiment_settings': {'metric': ['mse', 'mae', 'mape', 'rmse'],
                         'num_boost_round': 100,
                         'num_threads': 4,
                         'objective': 'regression',
                         'stopping_rounds': 100}}
